### Importar de Bibliotecas:

In [ ]:
from __future__ import print_function, division

%matplotlib inline
import matplotlib.pyplot as plt
from matplotlib.image import imread
import seaborn as sn
from IPython.display import Image, display
from PIL import Image as Imagem

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import torchvision
from torchvision import datasets, models, transforms
from torch.autograd import Variable

import numpy as np
import pandas as pd
import os
import time
import copy
import time
from datetime import timedelta

import tensorflow as tf
from tensorboardX import SummaryWriter


plt.ion()

In [ ]:
#### Verificação das Versões

In [ ]:
tf.__version__

In [ ]:
torch.__version__

#### Importar funções e classes provenientes dos arquivos inception.py e knifey.py
As funções e classes knifey e inception irão nos auxiliar ao tentarmos dividir o dataset em clusters

In [ ]:
import knifey

In [ ]:
import inception

In [ ]:
knifey.data_dir = "data/movedataset/"
data_dir = knifey.data_dir
dataset = knifey.load()

In [ ]:
class_names = dataset.class_names
class_names

In [ ]:
num_classes = len(class_names)

In [ ]:
image_paths_train, cls_train, labels_train = dataset.get_training_set()
image_paths_test, cls_test, labels_test = dataset.get_test_set()

In [ ]:
print (image_paths_train[0])
print (image_paths_test[0])

In [ ]:
print("Dimensões:")
print("- Dados de Treino:\t\t{}".format(len(image_paths_train)))
print("- Dados de Teste:\t\t{}".format(len(image_paths_test)))

### Função Auxiliar para Mostrar as Imagens

Função usada para mostrar 9 imagens em um
a tabela 3x3 e escrever o valor real e a predição abaixo de cada imagem:

In [ ]:
def plot_images(images, cls_true, cls_pred=None, smooth=True):

    assert len(images) == len(cls_true)

    # Create figure with sub-plots.
    fig, axes = plt.subplots(3, 3)

    # Adjust vertical spacing.
    if cls_pred is None:
        hspace = 0.3
    else:
        hspace = 0.6
    fig.subplots_adjust(hspace=hspace, wspace=0.3)

    # Interpolation type.
    if smooth:
        interpolation = 'spline16'
    else:
        interpolation = 'nearest'

    for i, ax in enumerate(axes.flat):
        # There may be less than 9 images, ensure it doesn't crash.
        if i < len(images):
            # Plot image.
            ax.imshow(images[i],
                      interpolation=interpolation)

            # Name of the true class.
            cls_true_name = class_names[cls_true[i]]

            # Show true and predicted classes.
            if cls_pred is None:
                xlabel = "True: {0}".format(cls_true_name)
            else:
                # Name of the predicted class.
                cls_pred_name = class_names[cls_pred[i]]

                xlabel = "True: {0}\nPred: {1}".format(cls_true_name, cls_pred_name)

            # Show the classes as the label on the x-axis.
            ax.set_xlabel(xlabel)
        
        # Remove ticks from the plot.
        ax.set_xticks([])
        ax.set_yticks([])
    
    # Ensure the plot is shown correctly with multiple plots
    # in a single Notebook cell.
    plt.show()

### Função auxiliar para carregar as imagens

Essa função carrega alguns arquivos de imagens

In [ ]:

def load_images(image_paths):
    # Load the images from disk.
    images = [imread(path) for path in image_paths]

    # Convert to a numpy array and return it.
    return np.asarray(images)

In [ ]:
# Load the first images from the test-set.
images = load_images(image_paths=image_paths_test[0:9])

# Get the true classes for those images.
cls_true = cls_test[0:9]

# Plot the images and labels using our helper-function above.
plot_images(images=images, cls_true=cls_true, smooth=True)

#### Download do Modelo Inception

In [ ]:
inception.maybe_download()

In [ ]:
#### Carregar o Modelo Inception

In [ ]:
model = inception.Inception()

#### Calcular Valores de Transferência

Importar uma função auxiliar para manter os valores de transferência do modelo inception em cache.

In [ ]:
from inception import transfer_values_cache

Setar os caminhos para armazenar os dados de treino e de teste em cache (arquivo plk)

In [ ]:
file_path_cache_train = os.path.join(data_dir, 'inception-tcc-train.pkl')
file_path_cache_test = os.path.join(data_dir, 'inception-tcc-test.pkl')

In [ ]:
print("Processing Inception transfer-values for training-images ...")

# If transfer-values have already been calculated then reload them,
# otherwise calculate them and save them to a cache-file.
transfer_values_train = transfer_values_cache(cache_path=file_path_cache_train,
                                              image_paths=image_paths_train,
                                              model=model)

In [ ]:
print("Processing Inception transfer-values for test-images ...")

# If transfer-values have already been calculated then reload them,
# otherwise calculate them and save them to a cache-file.
transfer_values_test = transfer_values_cache(cache_path=file_path_cache_test,
                                             image_paths=image_paths_test,
                                             model=model)

In [ ]:
### Dimensão dos valores de transferência

In [ ]:
print('Train:')
transfer_values_train.shape

In [ ]:
print('Test:')
transfer_values_test.shape

#### Função Auxiliar para visualização dos valores de transferência

In [ ]:
def plot_transfer_values(i):
    print("Input image:")
    
    # Plot the i'th image from the test-set.
    image = imread(image_paths_test[i])
    plt.imshow(image, interpolation='spline16')
    plt.show()
    
    print("Transfer-values for the image using Inception model:")
    
    # Transform the transfer-values into an image.
    img = transfer_values_test[i]
    img = img.reshape((32, 64))

    # Plot the image for the transfer-values.
    plt.imshow(img, interpolation='nearest', cmap='Reds')
    plt.show()

In [ ]:
### Visualização dos valores de transferência de duas imagens do conjunto de teste

In [ ]:
plot_transfer_values(i=20)

In [ ]:
plot_transfer_values(i=15)

#### Analise dos valores de transferência usando PCA 
--- Principal Component Analysis---

In [ ]:
from sklearn.decomposition import PCA

In [ ]:
pca = PCA(n_components = 2)

In [ ]:
transfer_values = transfer_values_train

In [ ]:
cls = cls_train

In [ ]:
transfer_values.shape

In [ ]:
transfer_values_reduced = pca.fit_transform(transfer_values)

In [ ]:
transfer_values_reduced.shape

#### Função Auxiliar para mostrar a redução dos valores de transferencia:

In [ ]:
def plot_scatter(values, cls):
    # Create a color-map with a different color for each class.
    import matplotlib.cm as cm
    cmap = cm.rainbow(np.linspace(0.0, 1.0, num_classes))

    # Create an index with a random permutation to make a better plot.
    idx = np.random.permutation(len(values))
    
    # Get the color for each sample.
    colors = cmap[cls[idx]]

    # Extract the x- and y-values.
    x = values[idx, 0]
    y = values[idx, 1]

    # Plot it.
    print('Matplotlib Graphic:')
    plt.scatter(x, y, color=colors, alpha=0.5)
    print('Seaborn Graphic:')
    g = sn.jointplot(x, y, kind='reg', scatter = False )
    g.ax_joint.scatter(x,y, c=colors)
    

In [ ]:
plot_scatter(transfer_values_reduced,cls=cls)

#### Analise dos valores de transformação usando t-SNE

In [ ]:
from sklearn.manifold import TSNE

In [ ]:
pca = PCA(n_components=50)
transfer_values_50d = pca.fit_transform(transfer_values)

In [ ]:
tsne = TSNE(n_components=2)

In [ ]:
transfer_values_reduced = tsne.fit_transform(transfer_values_50d)

In [ ]:
transfer_values_reduced.shape

In [ ]:
plot_scatter(transfer_values_reduced,cls=cls)

A partir dos resultados vistos nos gráficos acima é possível determinar que o modelo poderá ser treinado para obter resultados satisfatórios, visto que conseguimos separar as imagens em clusters bem definidos.

In [ ]:
#### Novo Classificador em Tensorflow

In [ ]:
#Definição dos logs de treino, validação e teste para acompanhamento via Tensorboard
log_writer_train = SummaryWriter('logs_ft/train/')
log_writer_val = SummaryWriter('logs_ft/val/')
#log_writer_test = SummaryWriter('logs/test/')

In [ ]:
# Normalização dos dados para treinamento
# Normalização para a etapa de validação

In [ ]:
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

In [ ]:
data_dir = 'data/dataset'
image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x),
                                          data_transforms[x])
                  for x in ['train', 'val']}
dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=4,
                                             shuffle=True, num_workers=4)
              for x in ['train', 'val']}
dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val']}
class_names = image_datasets['train'].classes

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [ ]:
dataloaders['train']

In [ ]:
def imshow(inp, title=None):
    """Imshow for Tensor."""
    inp = inp.numpy().transpose((1, 2, 0))
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    inp = std * inp + mean
    inp = np.clip(inp, 0, 1)
    plt.imshow(inp)
    if title is not None:
        plt.title(title)
    plt.pause(0.001)  # pause a bit so that plots are updated


# Get a batch of training data
inputs, classes = next(iter(dataloaders['train']))

# Make a grid from batch
out = torchvision.utils.make_grid(inputs)

imshow(out, title=[class_names[x] for x in classes])

In [ ]:
def train_model(model, criterion, optimizer, scheduler, num_epochs=25):
    since = time.time()
    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0
    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)
        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode
            running_loss = 0.0
            running_corrects = 0
            # Iterate over data.
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)
                # zero the parameter gradients
                optimizer.zero_grad()
                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)
                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()
                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]
            if phase == 'train':
                scheduler.step()
                log_writer_train.add_scalar('Loss',float(epoch_loss), epoch+1)
                log_writer_train.add_scalar('Accuracy', float(epoch_acc), epoch+1)
            elif phase =='val':
                log_writer_val.add_scalar('Loss',float(epoch_loss), epoch+1)
                log_writer_val.add_scalar('Accuracy', float(epoch_acc), epoch+1)
            print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                phase, epoch_loss, epoch_acc))
            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())
        
        print()
    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model

In [ ]:
def visualize_model(model, num_images=6):
    was_training = model.training
    model.eval()
    images_so_far = 0
    fig = plt.figure()

    with torch.no_grad():
        for i, (inputs, labels) in enumerate(dataloaders['val']):
            inputs = inputs.to(device)
            labels = labels.to(device)

            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)

            for j in range(inputs.size()[0]):
                images_so_far += 1
                ax = plt.subplot(num_images//2, 2, images_so_far)
                ax.axis('off')
                ax.set_title('predicted: {}'.format(class_names[preds[j]]))
                imshow(inputs.cpu().data[j])

                if images_so_far == num_images:
                    model.train(mode=was_training)
                    return
        model.train(mode=was_training)

In [ ]:
model_ft = models.inception_v3(pretrained=True)
model_ft.aux_logits=False
num_ftrs = model_ft.fc.in_features
# Here the size of each output sample is set to 2.
# Alternatively, it can be generalized to nn.Linear(num_ftrs, len(class_names)).
model_ft.fc = nn.Linear(num_ftrs, 2)

model_ft = model_ft.to(device)

criterion = nn.CrossEntropyLoss()

# Observe that all parameters are being optimized
optimizer_ft = optim.SGD(model_ft.parameters(), lr=0.001, momentum=0.9)

# Decay LR by a factor of 0.1 every 5 epochs
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=5, gamma=0.1)

In [ ]:
model_ft = train_model(model_ft, criterion, optimizer_ft,
                         exp_lr_scheduler, num_epochs=30)

In [ ]:
visualize_model(model_ft)

plt.ioff()
plt.show()

In [ ]:
#### Salvar o Modelo

In [ ]:
torch.save(model_ft, 'prediction_ft_model.pth')

In [ ]:
#### Add o graph no Tensorboard

In [ ]:
#### Inferência sobre o modelo criado

In [ ]:
model=torch.load('prediction_ft_model.pth')

In [ ]:
model.eval()

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
def process_image(image_path):
    # Load Image
    img = Imagem.open(image_path)
    
    # Get the dimensions of the image
    width, height = img.size
    
    # Resize by keeping the aspect ratio, but changing the dimension
    # so the shortest size is 255px
    img = img.resize((255, int(255*(height/width))) if width < height else (int(255*(width/height)), 255))
    
    # Get the dimensions of the new image size
    width, height = img.size
    
    # Set the coordinates to do a center crop of 224 x 224
    left = (width - 224)/2
    top = (height - 224)/2
    right = (width + 224)/2
    bottom = (height + 224)/2
    img = img.crop((left, top, right, bottom))
    
    # Turn image into numpy array
    img = np.array(img)
    
    # Make the color channel dimension first instead of last
    img = img.transpose((2, 0, 1))
    
    # Make all values between 0 and 1
    img = img/255
    
    # Normalize based on the preset mean and standard deviation
    img[0] = (img[0] - 0.485)/0.229
    img[1] = (img[1] - 0.456)/0.224
    img[2] = (img[2] - 0.406)/0.225
    
    # Add a fourth dimension to the beginning to indicate batch size
    img = img[np.newaxis,:]
    
    # Turn into a torch tensor
    image = torch.from_numpy(img)
    image = image.float()
    return image

In [ ]:
def predict(image, model):
    # Pass the image through our model
    output = model.forward(image)
    
    # Reverse the log function in our output
    output = torch.exp(output)
    
    # Get the top predicted class, and the output percentage for
    # that class
    probs, classes = output.topk(1, dim=1)
    return probs.item(), classes.item()

In [ ]:
# Show Image
def show_image(image):
    # Convert image to numpy
    image = image.numpy()
    
    # Un-normalize the image
    image[0] = image[0] * 0.226 + 0.445
    
    # Print the image
    fig = plt.figure(figsize=(25, 4))
    plt.imshow(np.transpose(image[0], (1, 2, 0)))

In [ ]:
image = process_image("20191103_142524.jpg")

In [ ]:
top_prob, top_class = predict(image, model)

In [ ]:
show_image(image)

In [ ]:
print('Classe: ',class_names[top_class]  )

In [ ]:
data_dir_infer = './data/dataset/val'

In [ ]:
def get_random_images(num):
    data = datasets.ImageFolder(data_dir_infer, transform=test_transforms)
    classes = data.classes
    indices = list(range(len(data)))
    np.random.shuffle(indices)
    idx = indices[:num]
    from torch.utils.data.sampler import SubsetRandomSampler
    sampler = SubsetRandomSampler(idx)
    loader = torch.utils.data.DataLoader(data, 
                   sampler=sampler, batch_size=num)
    dataiter = iter(loader)
    images, labels = dataiter.next()
    return images, labels

In [ ]:
test_transforms = transforms.Compose([transforms.Resize(224),
                                      transforms.ToTensor(),
                                     ])

In [ ]:
images, labels = get_random_images(5)

In [ ]:
writer = SummaryWriter('./logs_ft')
writer.add_graph(model_ft, images)

In [ ]:
os.chdir('./data/dataset/test/conformes')
conformes = os.listdir('.')

In [ ]:
pred_conforme=[]
for conf in conformes:
    image = process_image(conf)
    top_prob, top_class = predict(image, model)
    pred_conforme.append(top_class)

In [ ]:
cont_conforme_in_conforme=0
cont_n_conforme_in_conforme=0
for i in pred_conforme:
    if i == 0:
        cont_conforme_in_conforme+=1
    else:
        cont_n_conforme_in_conforme+=1
print('conformes em conformes:',cont_conforme_in_conforme)
print('não conformes em conformes:',cont_n_conforme_in_conforme)

In [ ]:
os.chdir('../nao-conformes')
nao_conformes = os.listdir('.')

In [ ]:
pred_n_conforme=[]
for nconf in nao_conformes:
    image = process_image(nconf)
    top_prob, top_class = predict(image, model)
    pred_n_conforme.append(top_class)

In [ ]:
cont_nconforme_in_nconforme=0
cont_conforme_in_nconforme=0
for i in pred_n_conforme:
    if i == 1:
        cont_nconforme_in_nconforme+=1
    else:
        cont_conforme_in_nconforme+=1
print('não-conformes em não-conformes:',cont_nconforme_in_nconforme)
print('conformes em não-conformes:',cont_conforme_in_nconforme)

In [ ]:
array = [[cont_conforme_in_conforme,cont_n_conforme_in_conforme],
         [cont_conforme_in_nconforme,cont_nconforme_in_nconforme]]        
df_cm = pd.DataFrame(array)
f, ax = plt.subplots(figsize=(9, 6))
sn.set()
sn.heatmap(df_cm, annot=True,linewidths=.5,fmt="d",ax=ax, cmap='cool')
ax.set_xlabel('Predicted labels');ax.set_ylabel('True labels'); 
ax.set_title('Confusion Matrix'); 
ax.xaxis.set_ticklabels(['Conformes', 'Não-Conformes'],va="center"); ax.yaxis.set_ticklabels(['Conformes', 'Não-Conformes'],va="center");


In [ ]:
conf_test=[]
nconf_test=[]
for i in range(len(pred_conforme)):
    conf_test.append(0)
for i in range(len(pred_n_conforme)):
    nconf_test.append(1)

In [ ]:
from sklearn.metrics import classification_report

In [ ]:
print(classification_report(conf_test,pred_conforme))

In [ ]:
print(classification_report(nconf_test,pred_n_conforme))